In [5]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
from pathlib import Path
pd.options.display.float_format = "{:,.2f}".format

# =========================================================
# Config mapeos / alias de columnas y nombres de hojas
# (ajusta aquí si tu archivo usa encabezados distintos)
# =========================================================
XLSX_PATH = Path("db/vulnerabilidad.xlsx")

SHEET_PERSONAS = "Personas"
SHEET_UNIVERSO = "Universo Familiares"
SHEET_INGRESOS = "Ingresos"
SHEET_DEUDAS = "Deudas"

# Dtype for IDs → todo texto
DTYPE_DICT = {
    "identificacion": str,
    "ruc_empleador": str,
    "ced_padre": str,
    "ced_madre": str,
}

# Columnas esperadas (puedes mapear si difieren en tu xlsx)
COL_PERIODO = "periodo"
COL_IDENT = "identificacion"
COL_TIPO = "tipo"  # 'A' (Afluentes) o 'E' (Enrollment)

COL_CED_PADRE = "ced_padre"
COL_CED_MADRE = "ced_madre"

COL_ANIO = "anio"
COL_MES = "mes"
COL_SALARIO = "salario"  # ingreso mensual
COL_DEUDA = "valor"  # deuda

# Qué meses usar para ingresos y deudas (por defecto 2025-06)
ANIO_FILTRO = 2025
MES_FILTRO = 6

# Multiplicador para llevar salario mensual a anual
SALARIO_MESES_ANO = 14


# =========================================================
# Helpers
# =========================================================


def parse_monto(val) -> float:
    if pd.isna(val):
        return float("nan")
    s = str(val).strip()
    if s == "":
        return float("nan")
    # quita símbolos y cambia coma→punto
    s = s.replace(" ", "").replace("$", "").replace(",", ".")
    try:
        return float(s)
    except Exception:
        return float("nan")


def _norm_cols(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [str(c).strip() for c in df.columns]
    return df


def make_hogar_id(ced_padre: str, ced_madre: str) -> str:
    """
    Genera un ID de hogar estable:
    - Si ambos '0' -> ""
    - Si solo uno válido -> ese
    - Si ambos válidos -> concatenación ordenada "menor-mayor"
    """
    a = str(ced_padre or "").strip()
    b = str(ced_madre or "").strip()
    a = "0" if a == "" else a
    b = "0" if b == "" else b
    if a == "0" and b == "0":
        return ""
    if a == "0":
        return b
    if b == "0":
        return a
    return "-".join(sorted([a, b]))


def _stats_from_series(s: pd.Series) -> dict:
    """Devuelve min, max, mediana, media, std y n para una serie numérica."""
    s = pd.to_numeric(s, errors="coerce")
    s = s.replace([np.inf, -np.inf], np.nan).dropna()
    return {
        "min": float(s.min()) if not s.empty else np.nan,
        "max": float(s.max()) if not s.empty else np.nan,
        "mediana": float(s.median()) if not s.empty else np.nan,
        "media": float(s.mean()) if not s.empty else np.nan,
        "desviacion": float(s.std(ddof=1)) if len(s) > 1 else 0.0,
        "n": int(s.size),
    }


def _remove_outliers_iqr_series(s: pd.Series, k: float = 1.5) -> pd.Series:
    """Filtra outliers de una serie usando Tukey (IQR)."""
    s = pd.to_numeric(s, errors="coerce").dropna()
    if s.empty:
        return s
    q1, q3 = s.quantile(0.25), s.quantile(0.75)
    iqr = q3 - q1
    low, high = q1 - k * iqr, q3 + k * iqr
    return s[(s >= low) & (s <= high)]


# =========================================================
# Carga de datos desde el Excel
# =========================================================
def cargar_vulnerabilidad_xlsx(path: Path) -> dict:
    # leemos todas las hojas, forzando columnas de ID a str
    wb = pd.read_excel(
        path,
        sheet_name=None,
        engine="openpyxl",
        dtype=DTYPE_DICT,  # 👈 aquí la normalización
    )
    # Normalizamos encabezados
    data = {name: _norm_cols(df) for name, df in wb.items()}
    return data


# =========================================================
# ETL para familias, salarios, deudas (replicando tu lógica)
# =========================================================
def obtener_datos_familias(
    dfs: dict, periodo: str, grupo_seleccionado: str
) -> pd.DataFrame:
    """
    Une Personas (filtrado por periodo y tipo) con Universo Familiares (padre/madre).
    Filtra filas donde al menos un padre/madre sea válido (distinto de '0').
    """
    df_personas = dfs.get(SHEET_PERSONAS, pd.DataFrame()).copy()
    df_universo = dfs.get(SHEET_UNIVERSO, pd.DataFrame()).copy()
    if df_personas.empty or df_universo.empty:
        return pd.DataFrame()

    # Normaliza algunas columnas clave a string
    for c in [COL_PERIODO, COL_IDENT, COL_TIPO]:
        if c in df_personas.columns:
            df_personas[c] = df_personas[c].astype(str).str.strip()

    # Filtra por periodo y grupo (tipo)
    personas_periodo = df_personas[
        (df_personas[COL_PERIODO] == str(periodo))
        & (df_personas[COL_TIPO] == str(grupo_seleccionado))
    ].copy()
    if personas_periodo.empty:
        return pd.DataFrame()

    # Normaliza universo familiares
    for c in [COL_IDENT, COL_CED_PADRE, COL_CED_MADRE]:
        if c in df_universo.columns:
            df_universo[c] = df_universo[c].astype(str).str.strip().replace({"": "0"})

    familias = personas_periodo.merge(df_universo, on=COL_IDENT, how="left")
    familias = familias[
        (familias[COL_CED_PADRE].fillna("0") != "0")
        | (familias[COL_CED_MADRE].fillna("0") != "0")
    ].copy()
    return familias


def obtener_datos_salario_deuda_familia(
    dfs: dict, familias_df: pd.DataFrame
) -> pd.DataFrame:
    if familias_df.empty:
        return familias_df

    out = familias_df.copy()
    out["salario_familiar"] = 0.0
    out["deuda_familiar"] = 0.0

    # --- Ingresos (vectorizado) ---
    df_ingresos = dfs.get("Ingresos", pd.DataFrame()).copy()
    if not df_ingresos.empty:
        if "salario" in df_ingresos.columns:
            df_ingresos["salario"] = df_ingresos["salario"].apply(parse_monto).fillna(0.0)
        ingresos_mes = df_ingresos[
            (df_ingresos["anio"] == ANIO_FILTRO) & (df_ingresos["mes"] == MES_FILTRO)
        ].copy()

        if not ingresos_mes.empty:
            ing_por_id = ingresos_mes.groupby("identificacion", as_index=False)[
                "salario"
            ].sum()
            ing_map = dict(zip(ing_por_id["identificacion"], ing_por_id["salario"]))

            s_padre = out["ced_padre"].map(ing_map).fillna(0.0)
            s_madre = out["ced_madre"].map(ing_map).fillna(0.0)
            out.loc[:, "salario_familiar"] = (s_padre + s_madre) * SALARIO_MESES_ANO

    # --- Deudas (vectorizado) ---
    df_deudas = dfs.get("Deudas", pd.DataFrame()).copy()
    if not df_deudas.empty:
        if "valor" in df_deudas.columns:
            df_deudas["valor"] = df_deudas["valor"].apply(parse_monto).fillna(0.0)
        deudas_mes = df_deudas[
            (df_deudas["anio"] == ANIO_FILTRO) & (df_deudas["mes"] == MES_FILTRO)
        ].copy()

        if not deudas_mes.empty:
            deuda_por_id = deudas_mes.groupby("identificacion", as_index=False)[
                "valor"
            ].sum()
            deuda_map = dict(zip(deuda_por_id["identificacion"], deuda_por_id["valor"]))

            d_padre = out["ced_padre"].map(deuda_map).fillna(0.0)
            d_madre = out["ced_madre"].map(deuda_map).fillna(0.0)
            out.loc[:, "deuda_familiar"] = d_padre + d_madre

    return out


def _hogares_unicos_salario_deuda(familias_df: pd.DataFrame) -> pd.DataFrame:
    """
    Condensa a 1 registro por hogar: hogar_id, salario_familiar, deuda_familiar.
    """
    if familias_df.empty:
        return pd.DataFrame(columns=["hogar_id", "salario_familiar", "deuda_familiar"])

    tmp = familias_df.copy()
    tmp[COL_CED_PADRE] = tmp[COL_CED_PADRE].astype(str).str.strip().replace({"": "0"})
    tmp[COL_CED_MADRE] = tmp[COL_CED_MADRE].astype(str).str.strip().replace({"": "0"})

    tmp["hogar_id"] = tmp.apply(
        lambda r: make_hogar_id(r[COL_CED_PADRE], r[COL_CED_MADRE]), axis=1
    )
    tmp = tmp[tmp["hogar_id"] != ""]

    hogares = (
        tmp.groupby("hogar_id", as_index=False)[["salario_familiar", "deuda_familiar"]]
        .first()
        .copy()
    )

    hogares["salario_familiar"] = pd.to_numeric(
        hogares["salario_familiar"], errors="coerce"
    ).fillna(0.0)
    hogares["deuda_familiar"] = pd.to_numeric(
        hogares["deuda_familiar"], errors="coerce"
    ).fillna(0.0)
    hogares = hogares[
        (hogares["salario_familiar"] >= 0) & (hogares["deuda_familiar"] >= 0)
    ]
    return hogares


def calcular_estadisticas_ratios(
    df_hogares: pd.DataFrame, remover_outliers: bool = False, k_iqr: float = 1.5
) -> pd.DataFrame:
    """
    Espera un DF con columnas salario_familiar (anual) y deuda_familiar (total).
    Calcula stats para:
      1) ingreso_anual / deuda_total  (denominador > 0)
      2) deuda_total / ingreso_anual  (denominador > 0)
    """
    idx = ["min", "max", "mediana", "media", "desviacion", "n"]
    if df_hogares.empty:
        return pd.DataFrame(
            index=idx, columns=["ingreso_sobre_deuda", "deuda_sobre_ingreso"]
        )

    r1_mask = df_hogares["deuda_familiar"] > 0
    r2_mask = df_hogares["salario_familiar"] > 0

    r1 = (
        df_hogares.loc[r1_mask, "salario_familiar"]
        / df_hogares.loc[r1_mask, "deuda_familiar"]
    )
    r2 = (
        df_hogares.loc[r2_mask, "deuda_familiar"]
        / df_hogares.loc[r2_mask, "salario_familiar"]
    )

    if remover_outliers:
        r1 = _remove_outliers_iqr_series(r1, k=k_iqr)
        r2 = _remove_outliers_iqr_series(r2, k=k_iqr)

    stats_r1 = _stats_from_series(r1)
    stats_r2 = _stats_from_series(r2)

    out = pd.DataFrame(
        {
            "ingreso_sobre_deuda": stats_r1,
            "deuda_sobre_ingreso": stats_r2,
        }
    )
    return out


# =========================================================
# Ejecución: por grupo (A/E) y por periodo(s)
# =========================================================
def ejecutar_consola(
    grupo_seleccionado: str, remover_outliers: bool = True, k_iqr: float = 1.5
):
    """
    Imprime en consola las estadísticas para cada periodo disponible del grupo indicado.
    grupo_seleccionado: 'A' (Afluentes) o 'E' (Enrollment)
    """
    dfs = cargar_vulnerabilidad_xlsx(XLSX_PATH)
    df_personas = dfs.get(SHEET_PERSONAS, pd.DataFrame()).copy()
    if df_personas.empty:
        print("No se encontró la hoja 'Personas' o está vacía.")
        return

    # Periodos disponibles para el grupo
    mask_grupo = df_personas[COL_TIPO].astype(str).str.strip() == str(
        grupo_seleccionado
    )
    periodos = (
        df_personas.loc[mask_grupo, COL_PERIODO]
        .dropna()
        .astype(str)
        .drop_duplicates()
        .sort_values()
        .tolist()
    )
    if not periodos:
        print(f"No hay periodos para el grupo {grupo_seleccionado}.")
        return

    print(f"\n=== Grupo: {grupo_seleccionado} | Periodos: {', '.join(periodos)} ===")
    print(
        f"Parámetros ingresos/deudas usados: año={ANIO_FILTRO}, mes={MES_FILTRO}, salario_anual={SALARIO_MESES_ANO}x\n"
    )

    for periodo in periodos:
        familias = obtener_datos_familias(dfs, periodo, grupo_seleccionado)
        if familias.empty:
            print(f"[{periodo}] Sin familias.")
            continue

        familias_sd = obtener_datos_salario_deuda_familia(dfs, familias)
        hogares = _hogares_unicos_salario_deuda(familias_sd)

        stats = calcular_estadisticas_ratios(
            hogares, remover_outliers=remover_outliers, k_iqr=k_iqr
        )

        print(f"--- Periodo {periodo} ---")
        print(f"Total hogares únicos: {len(hogares):,}")
        print(stats.to_string())
        print()


# =========================================================
# MAIN (ajusta grupo según necesites)
# =========================================================
if __name__ == "__main__":
    # Ejecuta para Enrollment (E) y Afluentes (A) si quieres
    ejecutar_consola(grupo_seleccionado="E", remover_outliers=False, k_iqr=1.5)
    # ejecutar_consola(grupo_seleccionado="A", remover_outliers=True, k_iqr=1.5)


=== Grupo: E | Periodos: 202520 ===
Parámetros ingresos/deudas usados: año=2025, mes=6, salario_anual=14x

--- Periodo 202520 ---
Total hogares únicos: 12,731
            ingreso_sobre_deuda  deuda_sobre_ingreso
min                        0.00                 0.00
max                2,038,400.00               664.93
mediana                    0.73                 0.67
media                    356.22                 2.90
desviacion            23,680.17                12.23
n                     10,969.00             9,797.00

